# Draft

In [ ]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
import datetime
import tqdm

In [ ]:
N = 5
rand = np.random.RandomState(1)
a = np.round(rand.normal(5, 1.5, size=(N, N)),2)
a[np.eye(N)==1] = 0
a = -(a / a.max())
a

In [ ]:
class MyEnv(gym.Env):
    def __init__(self, env_config):
        self.n = env_config['n']
        self.a = env_config['a']
        self.action_space = gym.spaces.Discrete(self.n)
        self.observation_space = gym.spaces.Box(low=-2.0, 
                                                high=0.0, 
                                                shape=(self.n,self.n), dtype=np.float32)
    def reset(self, seed=None, options=None):
        self.obs = self.a.copy()
        return self.obs, {}
        
    def step(self, action):
        reward = self.obs[0, action]
        self.obs[:, action] = -2.0
        terminated = False
        truncated = False
        if np.all(self.obs[0]<=-2.0):
            terminated = True
        else:
            # переставляем
            pass
        #print(f"action = {action} reward = {reward} terminated = {terminated}")
        return self.obs, reward, terminated, truncated, {}

In [ ]:
env = MyEnv(env_config = {"n": N, "a": a} )

In [ ]:
print(datetime.datetime.now())
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=1024, progress_bar = True)
print(datetime.datetime.now())

In [ ]:
obs, _ = env.reset()
actions = []
for i in range(100):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    actions.append(int(action))
    if terminated or truncated:
        print(f"{actions}")
        break
env.close()